In [1]:
import time
import scipy
import matplotlib.pyplot as plt
import matplotlib
import sys
import itertools
import json
import numpy as np
from odor_tracking_sim import utility
from pompy import models
# from matplotlib.widgets import Slider,Button
# from matplotlib.transforms import Bbox
from extras import UpdatingVPatch,plot_wedges
from core_functions import f0,f1,f1_wedge,f2,f3,f4,f5
from ipywidgets import interactive
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

In [25]:
#Constants that don't change with drag bars
num_flies = 20000
fly_speed = 1.6

number_sources = 8
radius_sources = 1000.0
source_locations, _ = utility.create_circle_of_sources(number_sources,
                radius_sources,None)
source_pos = scipy.array([scipy.array(tup) for tup in source_locations])
release_location = np.zeros(2)

intended_heading_angles = np.random.uniform(0,2*np.pi,num_flies)
intended_heading_angles = np.linspace(0,2*np.pi,num_flies)

initial_cone_angle = np.radians(10.)

In [28]:
windmag_slider = widgets.FloatSlider(
    value=1.,
    min=0,
    max=4.0,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

cone_angle_slider = widgets.FloatSlider(
    value=np.degrees(initial_cone_angle),
    min=0.,
    max=40.0,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

toggle = widgets.ToggleButton(
    value=False,
    description='Toggle',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description')


def f(wind_mag,toggle,cone_angle):

    wind_angle = 7*scipy.pi/8.
    # wind_mag = 1.0
  
    # cone_angle = np.radians(20.)
    cone_angle = np.radians(cone_angle)

    release_times=0.

    K = -.4
    x_0 = 300

      #Convert intended heading angles to track heading angles
    track_heading_angles,dispersing_speeds = f0(intended_heading_angles,wind_mag,
        wind_angle)

    intersection_distances,dispersal_distances = f1_wedge(
    track_heading_angles,source_pos,wind_angle,cone_angle)
    success_probabilities = f2(intersection_distances,K,x_0,source_pos,wind_angle)
    plume_assignments = f3(success_probabilities,dispersal_distances)
    dispersal_travel_times,release_to_chosen_plume_distances = f4(
            plume_assignments,dispersal_distances,dispersing_speeds)
  
    fig1 = plt.figure(1)
    ax = plt.subplot()
    
    xlim = (-1500., 1500.)
    ylim = (-1500., 1500.)
    im_extents = xlim[0], xlim[1], ylim[0], ylim[1]

    ax.set_ylim(list(ylim))
    ax.set_xlim(list(xlim))
    
    if toggle:
        plt.scatter(release_to_chosen_plume_distances*np.cos(track_heading_angles),
            release_to_chosen_plume_distances*np.sin(track_heading_angles),color='r',alpha=0.02)
        
    else: 
        time = 5*60.
        mag = time*dispersing_speeds 
        plt.scatter(mag*np.cos(track_heading_angles),
            mag*np.sin(track_heading_angles),color='r',alpha=0.02)
    
    wedge_points = plot_wedges(source_pos,wind_angle,cone_angle)

    plume_wedges = [matplotlib.patches.Polygon(
        wedge_points[:,i,:],color='black',alpha=0.2) for i in range(number_sources)]

    for plume_wedge in plume_wedges:
        ax.add_patch(plume_wedge)

    for x,y in source_locations:
        plt.scatter(x,y,marker='x',s=50,c='k')
    
    ax.set_aspect('equal')
    
    plt.xticks([])
    plt.yticks([])
    
    
interactive(f, wind_mag=windmag_slider,cone_angle=cone_angle_slider,toggle=toggle)
    

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='wind_mag', max=4.0, readout…

In [13]:
#     def update(val):

#         print(val)

#         wind_mag = windmag_slider.val
#         K = -1.
#         x_0 = 0.4
#         cone_angle = np.radians(10)
#         track_heading_angles,dispersing_speeds = f0(intended_heading_angles,wind_mag,wind_angle)
#         fly_dots.set_offsets(
#             scipy.c_[
#                 time*dispersing_speeds*np.cos(track_heading_angles),
#                 time*dispersing_speeds*np.sin(track_heading_angles)])
#         fig1.canvas.draw_idle()

In [13]:
button = widgets.Button(description="Toggle")
def on_button_clicked(b):
    for c in cb_container.children:
        if not c.value:
            plots.set_visible(False)
        else:
            plots.set_visible(True)
    plt.legend()
    plt.draw()
 
button.on_click(on_button_clicked)



interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='wind_mag', max=4.0, readout…

In [10]:

int_range = widgets.IntSlider()

def plot_state(change):
    print('b')
    pass


int_range.observe(on_value_change, names='value')
display(int_range)


IntSlider(value=0)